# Finding Critical Points of Neural Networks with Newton's Method

Attempt to reproduce the findings in
[Dauphin et al., 2014 - Identifying and attacking the saddle point problem in
high-dimensional non-convex optimization](https://arxiv.org/pdf/1406.2572.pdf).
Thus far unsuccessful.

We find that the gradient norms just don't go down very much when you use Newton's method to try to find critical points.

Differences:
- SGD instead of saddle-free newton
- no uniform noise
- different (larger) range of "fudging" values
- on a subset of MNIST

Unknowns:
- how did they downsample, and how much?
- what was their criterion for improvement?
- what was their stopping criterion?

In [1]:
from crit_finder.graphs import nn
from crit_finder import preprocess
from crit_finder import run

import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
subsampled_mnist = preprocess.subsample(mnist)

In [4]:
train_images, train_labels, test_images, test_labels = preprocess.get_train_and_test(mnist)

In [5]:
subsampled_mnist = preprocess.subsample(mnist)
downscaled_subsampled_mnist = preprocess.downscale(subsampled_mnist)

In [6]:
input_size = downscaled_subsampled_mnist["train"]["images"].shape[1]
output_size = 10

hyperparameters = {"layer_sizes":[10],
                            "nonlinearity":tf.nn.relu,
                            "input_size":input_size,
                            "output_size":output_size,
                             "learning_rate":0.5,
                             "inverse_method": "fudged"
                            }

net = nn.make(hyperparameters)
batch_size = 100

In [7]:
final_gd_parameters = run.gradient_descent(net, downscaled_subsampled_mnist, num_steps=1000, batch_size=batch_size)

gradient_norm	cost
0.07456606 2.3000405
0.14225022 1.1296964
0.060339503 0.6503058
0.040926628 0.5306064
0.031636793 0.46485096
0.024651445 0.42604536
0.020336146 0.40166166
0.018573852 0.38494128
0.019730171 0.3724223
0.024139566 0.3623992


In [8]:
best_parameters = run.optimally_fudged_newton_method(net, downscaled_subsampled_mnist, num_steps=10, batch_size=1000,
                                   initial_parameters = final_gd_parameters,
                                   fudge_factors = np.logspace(1,-5, num=7, dtype=np.float32))

0
0.03230041
	 new norm is 0.024071678519248962
	 new norm is 0.01293103862553835
	 new norm is 0.012686979956924915
1
0.012686984
	 new norm is 0.012591917999088764
	 new norm is 0.012429138645529747
	 new norm is 0.012080361135303974
2
0.012080359
	 new norm is 0.012058567255735397
	 new norm is 0.01200241968035698
	 new norm is 0.01180065143853426
3
0.01180065
	 new norm is 0.01179532054811716
	 new norm is 0.011767582036554813
4
0.011767582
	 new norm is 0.011747721582651138
	 new norm is 0.011714240536093712
5
0.011714239
	 new norm is 0.011685273610055447
	 new norm is 0.011630612425506115
6
0.011630609
	 new norm is 0.011626535095274448
7
0.011626532
	 new norm is 0.011623182334005833
8
0.0116231805
	 new norm is 0.011620264500379562
9
0.011620266
	 new norm is 0.011617626994848251
